In [1]:
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
import pandas as pd


In [19]:

# Load the Credit Card Fraud dataset
# data = np.loadtxt("creditcard.csv", skiprows=1, delimiter=",")
data = pd.read_csv("creditcard.csv")


In [22]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [34]:
drop_columns=['Time']
data.drop(columns=drop_columns, inplace=True)


In [35]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [43]:
data.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,...,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000,10001.000000
mean,-0.242155,0.282028,0.906234,0.264273,-0.046438,0.133053,-0.071727,-0.064698,0.802334,-0.222947,...,-0.052023,-0.152609,-0.033189,0.021370,0.087157,0.108105,0.005569,0.002919,63.025184,0.003800
std,1.521885,1.308097,1.159101,1.441217,1.182883,1.307257,1.077383,1.259026,1.155193,1.093505,...,0.913771,0.631082,0.487854,0.594410,0.428151,0.562775,0.410879,0.266235,184.477612,0.061527
min,-27.670569,-34.607649,-15.496222,-4.657545,-32.092129,-23.496714,-26.548144,-23.632502,-6.329801,-13.193415,...,-11.468435,-8.527145,-15.144340,-2.512377,-2.577363,-1.338556,-7.976100,-3.509250,0.000000,0.000000
25%,-1.013287,-0.208335,0.412846,-0.614358,-0.643270,-0.629894,-0.542309,-0.190711,0.070922,-0.688310,...,-0.268167,-0.549611,-0.174117,-0.327777,-0.158137,-0.327910,-0.084481,-0.015753,5.000000,0.000000
50%,-0.372973,0.288668,0.944342,0.219870,-0.153014,-0.152621,-0.055776,0.012940,0.805293,-0.340703,...,-0.123286,-0.136735,-0.045777,0.079996,0.121136,0.042850,-0.004566,0.015901,15.950000,0.000000
75%,1.150860,0.902053,1.602749,1.125899,0.370861,0.505266,0.476201,0.274678,1.506430,0.174445,...,0.032699,0.247584,0.081757,0.410813,0.359035,0.475996,0.120885,0.077180,50.950000,0.000000
max,1.960497,8.636214,4.101716,10.463020,34.099309,21.393069,34.303177,5.060381,10.392889,12.259949,...,22.588989,4.534454,13.876221,3.200201,5.525093,3.517346,8.254376,4.860769,7712.430000,1.000000


In [42]:
data=data.loc[:10000]

In [61]:
data['Class'].value_counts()

Class
0    9963
1      38
Name: count, dtype: int64

In [44]:

X = data.drop('Class', axis=1)
y = data['Class']


In [45]:
# Split the dataset into training data and test data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42) 


In [58]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


In [ ]:
# Step 3: Preprocess the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_test)


In [54]:

# Step 4: Implement SVM algorithm on training data and tune parameters
# Adjust the values of kernel, gamma, and nu as needed
svm = OneClassSVM(kernel='rbf', gamma=0.5, nu=0.5)
svm.fit(X_train_scaled)


OneClassSVM(gamma=0.5)

In [55]:

# Step 5: Make outlier predictions on the validation dataset
y_pred = svm.predict(X_val_scaled)


In [60]:

# Calculate metrics
accuracy_tree = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, pos_label=-1)
# recall = recall_score(y_test, y_pred, pos_label=-1)
# f1 = f1_score(y_test, y_pred, pos_label=-1)


# Print the metrics
print("accuracy_tree:", accuracy_tree)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)


accuracy_tree: 0.0


In [8]:

# Train the one-class SVM model
clf = OneClassSVM(kernel="linear", gamma=0.01, nu=0.95)
clf.fit(X_train)


OneClassSVM(gamma=0.01, kernel='linear', nu=0.95)

In [9]:
from sklearn.metrics import precision_score, recall_score
# Make outlier predictions on the validation dataset
y_pred = clf.predict(X_test)

# Calculate the precision, recall, and F1 score
# precision = np.sum(y_pred[y_val == -1] == -1) / np.sum(y_pred == -1)
# recall = np.sum(y_pred[y_val == -1] == -1) / np.sum(y_val == -1)


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [17]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
precision = precision_score(y_test, y_pred, average='macro')
# recall = recall_score(y_test, y_pred)
# f1 = 2 * precision * recall / (precision + recall)
accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred) 
# print("Precision:", precision)
# print("Recall:", recall)
print("F1 score:", precision)


F1 score: 0.00045866299736268775


c:\Users\Alireza Nasoodi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
